### <p style="text-align: right;"> &#9989; Andrew Koren</p>

# PHY480 Day 23

## In-class assignment: Molecular dynamics simulations

In this in-class assignment we incorporate forces in the molecular dynamics simulations as well as modify the initialization functions.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit
%matplotlib inline


**Task 1.** Implement computation of forces for Lennard-Jones potential (set to zero for $r>r_{cut}$). Compute them also by hand for the three particles given below with periodic boundary conditions and check that your code produces the same result. Take into account that some particles are closer through the boundary.


In [160]:
### main parameters

# dimension
dim = 2

# number of particles
N = 3

# particle radius (for later plotting)
radius = 0.5

# set potential to zero at this distance
rcut = 2.5

# box sizes (the size of this array should match "dim"),
# to simplify, box size should be >= 2*rcut
lbox = np.array( [5.0,5.0] )

# allocate position array
r = np.zeros( (N,dim) )

# allocate array for the forces
f = np.zeros( (N,dim) )

# set fixed positions
r[0,0], r[0,1] = 0.5, 2
r[1,0], r[1,1] = 4, 1
r[2,0], r[2,1] = 2, 3.5


In [ ]:
# compute force on every particle due to all other particles,
# Lennard-Jones potential 4(1/r^12-1/r^6)
# Input:
# lbox -- array with sizes
# r -- allocated array of shape N * dim,
#      where N is the number of particles and dim is the dimensionality
# f -- array for forces, same shape as r
# rcut -- maximum distance for the Lennard-Jones potential (set to 0 beyond)
# PBC -- =True if periodic boundary conditions are used, =False for hardwall boundary conditions
# Output:
# the input array f is changed
@njit
def forces_lj( lbox, r, f, rcut, PBC ):

    # set the parameters for calculation
    n = r.shape[0]  # read off the number of particles
    nd = r.shape[1] # read off the dimensions
    rcut_sq = rcut*rcut # to speed up comparisons #whats this for?
    dr = np.zeros(nd) # vector between two particles
    df = np.zeros(nd) # force contribution from one pair
    lbox2 = lbox/2 # speed up calculations
    
    # set the force array to zero
    f *= 0
    
    # loop over the first particle
    for i in range(N):
        # loop over the second particle
        for j in range(i+1, N):
            Dx = r[j] - r[i]
            if PBC:
                Dx = (Dx + lbox2) % lbox - lbox2

            # if dist_sq is less than rcut_sq, there is a contribution to the force from this pair
            dist = np.sqrt(np.sum(Dx**2))
            if dist < rcut:
                continue
            
            df = 24/dist**8 * ( 1 - 2/dist) * Dx
            f[i] += df
            f[j] -= df


**Task 2.** Implement computation of the potential energy, i.e. summing up the Lennard-Jones potential for all pairs within $r_{cut}$. Compute it also by hand for the same three-particle arrangement with periodic boundary conditions and check that your code produces the same result.

In [35]:
# compute potential energy for particles interacting with
# Lennard-Jones potential 4(1/r^12-1/r^6)
# Input:
# lbox -- array with sizes
# r -- allocated array of shape N * dim,
#      where N is the number of particles and dim is the dimensionality
# PBC -- =True if periodic boundary conditions are used, =False for hardwall boundary conditions
# Output:
# potential energy
def measure_pot( lbox, r, rcut, PBC=True ):
    
    # read off parameters
    n = r.shape[0]
    nd = r.shape[1]
    rcut_sq = rcut*rcut
    dr = np.zeros(nd)
    lbox2 = lbox/2

    pot = 0
    
    # loop over the first particle
    for i in range(N):
        # loop over the second particle
        for j in range(i+1, N):
            Dx = r[j] - r[i]
            if PBC:
                Dx = (Dx + lbox2) % lbox - lbox2
            
            dist = np.sqrt(np.sum(Dx**2))
            if dist < rcut:
                continue    

            pot += 4 * (dist**(-12) - dist**(-6))


    return pot


**Task 3.** Implement computation of the kinetic energy, i.e. summing $m_iv_i^2/2$ for all particles.


In [36]:
# compute the kinetic energy -- sum of mv^2/2 for all particles
# Input:
# v -- velocities, N*dim array
# masses -- arrya of size N with particle masses
def measure_kin( v, masses ):

    v2 = np.sum(Dx**2)
    
    return sum(v2*masses/2)


**Task 4.** Modify the function that generates random particle positions (we already built one in the previous in-class assignment) in the following way:

- loop over particles ($N$ in total),
- generate a random particle position,
- loop over the particles that you have already generated and check if the new particle is at distance $r\geqslant1$ away from every other particle,
- if that is not the case, discard and regenerate a new random position for that particle,
- repeat the process until an acceptable random position is found,
- add this particle position to the `r` array and proceed with the next particle.

This method will insure that the initial particle configuration cannot produce instabilities (i.e. large forces).


In [ ]:
from scipy.spatial.distance import cdist

# initialize particle positions
# Input:
# lbox -- array with sizes
# r -- allocated array of shape N * dim,
#      where N is the number of particles and dim is the dimensionality
# Output:
# the input r is changed
def initial_r( lbox, r ):
    N, dim = r.shape

    r[:] = np.random.rand(*r.shape) * lbox
    for i in range(1,N):
        while (cdist(r[:N+1],r[:N+1]) + np.eye(len(r[:N+1]), len(r[:N+1]))*2 < 1).any():
            r[i] = np.random.rand(2) * lbox



**Task 5.** Modify the function that generates the initial velocities. The components of the velocity for each particle $i$ should be drawn from a Gaussian distribution with zero mean and $\sqrt{T/m_i}$ standard deviation, where $T$ is the temperature and $m_i$ is the $i$-th particle mass. Once all the velocities are generated, compute the velocity of the center of mass (i.e. compute the total momentum and divide by the total mass). Then subtract the center-of-mass velocity from the velocity of every particle.

In [322]:
# initialize particle velocities, subtract the velocity of the center of mass
# Input:
# lbox -- array with sizes
# v -- allocated array of shape N * dim,
#      where N is the number of particles and dim is the dimensionality
# Output:
# the input v is changed
def initial_v( lbox, v, T, masses ):
    masses = np.array([[m] for m in masses])
    sigma = np.sqrt(T/masses)

    v[:] = np.random.randn(3,2)  * sigma
    
    total_momentum = np.sum(masses * v, axis=1, keepdims=True)
    total_mass = np.sum(masses)
    
    v[:] -= total_momentum / total_mass

<div style="page-break-after: always"></div>

**Task 7.** Modify the update functions for the position and velocity to incorporate the forces.


In [9]:
# propagate the particle positions by step dt
# Input:
# dt -- time step
# r -- array with particle positions
# v -- array with particle velocities
# Output:
# the input r is changed
def update_r( dt, r, v ):
    
    # ...
    
    return


In [10]:
# propagate the particle velocities by step dt
# Input:
# dt -- time step
# v -- array with particle velocities
# f -- array with forces
# masses -- array with masses
# Output:
# the input v is changed
def update_v( dt, v, f, masses ):

    # ...

    return


**Task 8.** Using the boundary conitions function from the previous assignment, put together a full molecular dynamics simulation. In the outer `Nsim` loop of the simulation measure the kinetic, potential and total energy in the simulation. Plot the total energy.


In [11]:
### main parameters

# dimension
dim = 2

# number of particles
N = 10 # increase once the code is working properly

# particle radius (for plotting)
radius = 0.5

# set potential to zero at this distance
rcut = 2.5

# box sizes (the size of this array should match "dim"),
# to simplify, box size should be >= 2*rcut
lbox = np.array( [8.0,8.0] )

# allocate arrays for the position r and velocity v,
# the first index is particle index and the second is the vector component
r = np.zeros( (N,dim) )
v = np.zeros( (N,dim) )

# allocate array for the forces
f = np.zeros( (N,dim) )

# array with masses (set to 1 for now)
masses = np.full( N, 1.0 )


# for reproducibility
np.random.seed(2)

# time step
dt = 0.003

# temperature (in microcanonical distribution it is only used for the initial configuration)
Temp = 1.3

# number of simulation steps
Nsim = 20
Ninner = 1

# periodic boundary conditions if True
PBC = True



In [12]:
# initialize particle positions and velocities
initial_r( lbox, r )
initial_v( lbox, v, Temp, masses )


In [13]:
# precompute forces


# main simulation loop
    
    # inner loop

        # propagate velocities by dt/2

        # propagate positions r by dt

        # apply boundary conditions

        # compute forces

        # propagate velocities by dt/2
    
    # compute observables (kinetic, potential energy, etc.)

    # print info



&#169; Copyright 2025,  Michigan State University Board of Trustees